In [10]:
import torchaudio as ta
from chatterbox.tts import ChatterboxTTS
from chatterbox.mtl_tts import ChatterboxMultilingualTTS

# English example
model = ChatterboxTTS.from_pretrained(device="cpu")

# text = "Ezreal and Jinx teamed up with Ahri, Yasuo, and Teemo to take down the enemy's Nexus in an epic late-game pentakill."
# wav = model.generate(text)
# ta.save("test-english.wav", wav, model.sr)

# Multilingual examples
multilingual_model = ChatterboxMultilingualTTS.from_pretrained(device="cpu")

french_text = "السلام عليكم كيف الحال ؟ أنا نموذج لغوي صوتي "
wav_french = multilingual_model.generate(french_text, language_id="ar")
ta.save("test-french.wav", wav_french, model.sr)

/teamspace/studios/this_studio/chatterbox/.venv/lib/python3.11/site-packages/diffusers/models/lora.py:393: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)


loaded PerthNet (Implicit) at step 250,000


Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

loaded PerthNet (Implicit) at step 250,000


/system/conda/miniconda3/uv/python/cpython-3.11.13-linux-x86_64-gnu/lib/python3.11/contextlib.py:105: FutureWarning: `torch.backends.cuda.sdp_kernel()` is deprecated. In the future, this context manager will be removed. Please see `torch.nn.attention.sdpa_kernel()` for the new context manager, with updated signature.
  self.gen = func(*args, **kwds)
/teamspace/studios/this_studio/chatterbox/.venv/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:774: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_attentions` is. When `return_dict_in_generate` is not `True`, `output_attentions` is ignored.
  warnings.warn(
LlamaModel is using LlamaSdpaAttention, but `torch.nn.functional.scaled_dot_product_attention` does not support `output_attentions=True`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argume

## Voice Cloning

In [9]:
import torch
import torchaudio as ta
from datasets import load_dataset 
import numpy as np
import os
import tempfile # <-- New import for handling temporary files

from chatterbox.vc import ChatterboxVC

# --- 1. Setup Device and Model ---
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Using device: {device}")

# Load the Voice Conversion model
model = ChatterboxVC.from_pretrained(device)
# print(f"loaded PerthNet (Implicit) at step {model.ckpt_info['step']} ✅")

# --- 2. Load the Hugging Face Dataset ---
DATASET_NAME = "hamzabouajila/msa_law_asr_test"
try:
    dataset = load_dataset(DATASET_NAME, split="test")
    print(f"✅ Loaded dataset '{DATASET_NAME}' with {len(dataset)} samples.")
except Exception as e:
    print(f"❌ Could not load dataset '{DATASET_NAME}'. Error: {e}")
    exit()

# --- 3. Prepare Source Audio and Target Voice ---

# The target voice we want to clone is the dataset's first sample.
source_audio_dict = dataset[0]["file"] 

# The 'array' key holds the decoded NumPy array of the waveform
source_array = source_audio_dict["array"]

# Convert the NumPy array to a PyTorch tensor, ensuring float32 dtype
SOURCE_WAV_TENSOR = torch.tensor(source_array, dtype=torch.float32).unsqueeze(0).to(device)

# Get the sampling rate
SOURCE_SR = source_audio_dict["sampling_rate"]

# Check if the model's required sample rate matches the source sample rate
if SOURCE_SR != model.sr:
    print(f"⚠️ Resampling source audio from {SOURCE_SR}Hz to model's {model.sr}Hz.")
    resampler = ta.transforms.Resample(orig_freq=SOURCE_SR, new_freq=model.sr).to(device)
    SOURCE_WAV_TENSOR = resampler(SOURCE_WAV_TENSOR)

# --- 4. Save the Target Voice Tensor to a Temporary WAV File ---

# Create a temporary file path
TEMP_TARGET_VOICE_PATH = os.path.join(tempfile.gettempdir(), "temp_target_voice.wav")

# Save the PyTorch tensor (which is the voice we want to clone) to the temp path
ta.save(TEMP_TARGET_VOICE_PATH, SOURCE_WAV_TENSOR.cpu(), model.sr) 

# The path to the source audio is now a fixed path you provided:
SOURCE_AUDIO_PATH = "/teamspace/studios/this_studio/chatterbox/test-french.wav"

print(f"\nSource Audio Path: {SOURCE_AUDIO_PATH}")
print(f"Target Voice Path (from dataset): {TEMP_TARGET_VOICE_PATH}")
print(f"Source Audio Shape (after resampling): {SOURCE_WAV_TENSOR.shape} (Channels, Samples)")

# --- 5. Generate Voice Conversion ---

print("\n⚙️ Starting voice conversion...")
# Pass both paths (strings) to the generate method
wav = model.generate(
    audio=SOURCE_AUDIO_PATH,        # The audio content to speak
    target_voice_path=TEMP_TARGET_VOICE_PATH, # The voice to clone (as a path)
)

# --- 6. Save the Result ---
OUTPUT_FILENAME = "testvc.wav"
ta.save(OUTPUT_FILENAME, wav.cpu(), model.sr)

print(f"✅ Voice conversion complete. Output saved to '{OUTPUT_FILENAME}' (Sample Rate: {model.sr} Hz).")

# Clean up the temporary file
os.remove(TEMP_TARGET_VOICE_PATH)
print(f"🧹 Cleaned up temporary file: {TEMP_TARGET_VOICE_PATH}")

Using device: cpu


loaded PerthNet (Implicit) at step 250,000
✅ Loaded dataset 'hamzabouajila/msa_law_asr_test' with 20 samples.
⚠️ Resampling source audio from 16000Hz to model's 24000Hz.

Source Audio Path: /teamspace/studios/this_studio/chatterbox/test-french.wav
Target Voice Path (from dataset): /tmp/temp_target_voice.wav
Source Audio Shape (after resampling): torch.Size([1, 504000]) (Channels, Samples)

⚙️ Starting voice conversion...
✅ Voice conversion complete. Output saved to 'testvc.wav' (Sample Rate: 24000 Hz).
🧹 Cleaned up temporary file: /tmp/temp_target_voice.wav
